In [1]:
import pandas as pd
import glob
from matplotlib import pyplot as plt
import numpy as np
import plotly.express as px
import seaborn as sns 
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from myst_nb import glue
#from ipywidgets import widgets
#from ipywidgets import interactive
#from IPython.display import display, Javascript

from matplotlib.colors import LogNorm, Normalize
import bamboolib

In [2]:
import os
current_directory = os.getcwd()
#print("Current directory : ", current_directory)

aymeric =  "/home/aymeric/python-scripts/espadon/data/" #aymeric
jp = '~/Dropbox/Mac/Desktop/CRD Anses/all3/' # Jean Philippe
jp_index = '~/Dropbox/Mac/Desktop/CRD Anses/code/indexation_results/' # Jean Philippe index

if 'aymeric' in current_directory:
    path_base = aymeric

elif 'Mac' in current_directory:
    path_base = jp
elif 'd:/Projects' in current_directory:
    path_base = "d:/Projects/Medialab/"

#print("Path base : ", path_base)

In [3]:

dic_id={}
for x in [x for x in pd.read_csv(glob.glob(f'{path_base}sm/*.csv')[0]).columns if 'id' in x]:
    dic_id[x]=str



In [4]:
df0= pd.read_csv(f"{path_base}corpus_tweets.csv", sep = ",", dtype = dic_id)

In [5]:
url_checked = pd.read_csv('list_url.csv', sep = ",")

In [6]:
url_checked

                       short_url  \
0          http://bit.ly/2GNJNwa   
1       http://ow.ly/NL71304Nvpf   
2             http://ow.ly/XSulJ   
3       http://ow.ly/o5ea50GHL81   
4             http://ow.ly/8Uz54   
...                          ...   
182559        http://ow.ly/CNM8x   
182560        http://ow.ly/YNNVM   
182561     http://fb.me/yUojI7tx   
182562  http://ow.ly/lUpD30mgILE   
182563   https://buff.ly/3qSEsHb   

                                                 long_url  
0                                                  Failed  
1       https://ascopost.com/archive/meetings/2016-ast...  
2                                                  Failed  
3                                                  Failed  
4                                                  Failed  
...                                                   ...  
182559                                             Failed  
182560  https://ascopost.com/News/36333?platform=hoots...  
182561                     

In [7]:

#users = pd.read_csv('../outcome/user_sm_predicted.csv',dtype=dic_id) #jean-philippe
users = pd.read_csv(f'{path_base}recoded_user_sm_predicted.csv',dtype=dic_id)


In [8]:
df0.columns

Index(['Unnamed: 0', 'query', 'id', 'timestamp_utc', 'local_time',
       'user_screen_name', 'text', 'possibly_sensitive', 'retweet_count',
       'like_count', 'reply_count', 'lang', 'to_username', 'to_userid',
       'to_tweetid', 'source_name', 'source_url', 'user_location', 'lat',
       'lng', 'user_id', 'user_name', 'user_verified', 'user_description',
       'user_url', 'user_image', 'user_tweets', 'user_followers',
       'user_friends', 'user_likes', 'user_lists', 'user_created_at',
       'user_timestamp_utc', 'collected_via', 'match_query', 'retweeted_id',
       'retweeted_user', 'retweeted_user_id', 'retweeted_timestamp_utc',
       'quoted_id', 'quoted_user', 'quoted_user_id', 'quoted_timestamp_utc',
       'collection_time', 'url', 'place_country_code', 'place_name',
       'place_type', 'place_coordinates', 'links', 'domains', 'media_urls',
       'media_files', 'media_types', 'mentioned_names', 'mentioned_ids',
       'hashtags', 'intervention_type', 'intervention_tex

In [9]:
df0 = df0[['query', 'id', 'timestamp_utc', 'local_time',
           'user_screen_name', 'text',  'user_location',  'user_id', 'user_name',
           'domains', 'links', 'media_urls', 'url',
           'ROS1', 'ALK', 'EXON', 'EGFR', 'KRAS', 'NTRK',
           'BRAF', 'MET', 'RET', 'HER2', 'date', ]]

df0['date'] = pd.to_datetime(pd.to_datetime(df0['date']).dt.date)
df0['Year'] = df0['date'].dt.year


In [10]:
df=df0.merge(users,on=['user_id'], how = "inner")#how = inner by default
#df = df.loc[(df["User_status"] != "Other") & (df["User_status"] != "Undefined") & (df["Year"] >= 2012)]

In [11]:
df.columns

Index(['query', 'id', 'timestamp_utc', 'local_time', 'user_screen_name',
       'text', 'user_location', 'user_id', 'user_name_x', 'domains', 'links',
       'media_urls', 'url', 'ROS1', 'ALK', 'EXON', 'EGFR', 'KRAS', 'NTRK',
       'BRAF', 'MET', 'RET', 'HER2', 'date', 'Year', 'Unnamed: 0',
       'user_creat', 'user_descr', 'user_follo', 'user_frien', 'user_image',
       'user_likes', 'user_lists', 'user_locat', 'user_name_y', 'user_numbe',
       'user_scree', 'user_tweet', 'user_url', 'user_id_id', 'comment_text',
       'training_set', 'validation_set', 'predicted_categories_ML', 'advocacy',
       'cancer patient', 'collective', 'corpus', 'female', 'health center',
       'health professional', 'industry', 'male', 'media', 'npo', 'oncologist',
       'other', 'research', 'survivor', 'User_type', 'Gender', 'User_status',
       'User_role', 'User_role2', 'User_role3', 'User_status.1',
       'User_status2', 'User_status3'],
      dtype='object')

In [12]:
df1 = df.loc[df['links'].notna()]



In [13]:
list_tweet_id = []
list_links = []
is_doi = []
list_user_scr_name = []
list_user_name = []
list_date = []

for i, link in enumerate(df1["links"]):
    tweet_id = df1["id"].iloc[i]
    user_scr_name = df1["user_screen_name"].iloc[i]
    user_name = df1["user_name_x"].iloc[i]
    user_status = df1["User_status"].iloc[i]
    date = df1["date"].iloc[i]
    tweet_id = df1["id"].iloc[i]
    list_url = link.split("|")
    for url in list_url:
        if "twitter.com" not in url and "instagram" not in url and "facebook.com" not in url and "youtu" not in url:
            list_tweet_id.append(tweet_id)
            list_links.append(url)
            list_user_scr_name.append(user_scr_name)
            list_user_name.append(user_name)
            list_date.append(date)
            if "doi" in url:
                is_doi.append(True)
            else:
                is_doi.append(False)

KeyboardInterrupt: 

In [14]:
len(list_links)

586670

In [15]:
data = {"id": list_tweet_id, "urls": list_links, "is_doi": is_doi}
df_url = pd.DataFrame(data)

In [16]:
df_url1 = df_url.merge(df[["id", "user_screen_name", "date", "User_status"]], on = ["id"],
                       how = "inner")

In [17]:
df_url1

                         id                        urls  is_doi  \
0       1111398869982539776  https://prognosis.ucsf.edu   False   
1       1111398869982539776  https://prognosis.ucsf.edu   False   
2       1111398869982539776  https://prognosis.ucsf.edu   False   
3       1111398869982539776  https://prognosis.ucsf.edu   False   
4       1111398561214681090       http://www.mycarg.org   False   
...                     ...                         ...     ...   
588607  1361382525206167555     https://buff.ly/37gH2Pa   False   
588608  1360631081180008456     https://buff.ly/3aJwZD7   False   
588609  1359880085784952837     https://buff.ly/36THXou   False   
588610  1359595070794653706     https://buff.ly/3tEL5hB   False   
588611  1358811580977315842     https://buff.ly/3tB4UGF   False   

       user_screen_name       date    User_status  
0        kimberli_coxmd 2019-03-28     Oncologist  
1        kimberli_coxmd 2019-03-28     Oncologist  
2        kimberli_coxmd 2019-03-28     

In [18]:
list_urls = set(list_links)

In [19]:
len(list_urls)

395810

In [20]:
from ural import is_shortened_url
from ural import is_url

In [21]:
shortened_url_list = []

for x in list(list_urls):
    if is_shortened_url(x) == True:
        shortened_url_list.append(x)

In [22]:
len(shortened_url_list)

200435

In [23]:
true_url_list = []

for x in shortened_url_list:
    if is_url(x) == True:
        true_url_list.append(x)

In [24]:
short_url_list = [x for x in url_checked["short_url"]]

In [25]:
len(short_url_list)

135522

In [26]:
true_url_list2 = [x for x in true_url_list if x not in short_url_list]
len(true_url_list2)

64913

In [27]:
from urllib import request

In [28]:

r = request.urlopen("http://owl.li/Nxvz30j7XtU")
r.headers

In [29]:
from tqdm.notebook import tqdm, trange
import time
import csv

In [ ]:

start_time = time.time()
#print(start_time)
for n, x in  tqdm(enumerate(true_url_list2)):
    #print(n, x)
    try:
        r = request.urlopen(x, timeout =3)
        with open('list_url.csv', 'a', newline='') as csvfile:
            fieldnames = ['short_url', 'long_url']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writerow({'short_url': x, 'long_url': r.url})
    except:
        with open('list_url.csv', 'a', newline='') as csvfile:
            fieldnames = ['short_url', 'long_url']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writerow({'short_url': x, 'long_url': 'Failed'})
    


0it [00:00, ?it/s]